In [ ]:
import sys
import os
import numpy as np
import random
import time
import re

import cv2
import torch
import torchvision
import numpy.random as random
from torchvision import transforms
from torch.autograd import Variable
from torchvision import datasets

from torch import optim
from PIL import Image
import matplotlib.pyplot as plt
from torch.nn import init

import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torchvision.transforms.functional as TF
from cv2 import imread

import functools
import itertools
from math import exp
from functools import partial

In [ ]:
mode = 'train' #help=train or test
exp_name = 'horses2zebras' # name of the experiment
#exp_name = 'apples2oranges'
#exp_name = 'catsvsdogs'
#exp_name = 'cityscapes'
n_epochs = 200 #help=number of epochs of training
batchsize = 4 #help=size of the batches
lr = 0.0001 #help=adam: learning rate
b1 = 0.5 #help=adam: decay of first order momentum of gradient
b2 = 0.999 #help=adam: decay of first order momentum of gradient
n_cpu = 4 #help=number of cpu threads to use during batch generation
img_size = 256 #help=size of each image dimension
channels = 3 #help=number of image channels
img_shape = channels, img_size, img_size # can be manually set if needed
print(np.prod(img_shape),img_shape)
load = False # help='load file model' Useful for resuming training from a checkpoint
scale = 0.5 # help='downscaling factor of the images')
save_epoch_freq = 10 # help='frequency of saving checkpoints at the end of epochs'
dir_checkpoint = './checkpoints' # directory to save the checkpoints
pool_size = 32 * batchsize # help = set the pool size. Can be set to different sizes for different experiments

In [ ]:
init_seed = 0 
torch.manual_seed(init_seed)
np.random.seed(init_seed)

In [ ]:
cuda = True if torch.cuda.is_available() else False
if(cuda):
    torch.cuda.manual_seed(init_seed)
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True

In [ ]:
FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

In [ ]:
norm = 'batch' # the name of the normalization layer: batch | instance | none
init_type = 'normal' # the name of an initialization method: normal | xavier | kaiming | orthogonal
init_gain = 0.02 # scaling factor for normal, xavier and orthogonal.
lambda_criterionGAN = 10.0 # weight 
lambda_L1_real = 200.0 #
lambda_L1_fake = 10.0 # 

In [ ]:
def pil_loader(path):
    return Image.open(path).convert('RGB')

In [ ]:
# help = assumes that the dataset is available in the dir: ./data/'task'/ [train,test].pt
# use the helper function at the end of the notebook to generate dataset in the required format
def dataset_memory_loader(task, ops, dir_data  = './data/'):
    for op in ops:
        images = []
        file_names = []
        path = dir_data+task+'/'+op
        print(path)
        count = 0
        for file in os.listdir(path):
            img_path = path+'/'+file
            file_names.append(img_path)
            img = pil_loader(img_path)
            img_int = np.array(img,dtype = 'uint8')
            images.append(img_int)
        count = count + 1
        return file_names, images

In [ ]:
class generic_dataset(torch.utils.data.Dataset):
    #The dataset is saved as dictionary files on disks
    #Merge the dictionary files depending upon the size of RAM before passing in
    #Dictionary key: path of the file; item: Image file loaded as uint8 bytes using PIL Image
    
    import numpy as np
    import re
    
    def __init__(self, data_dict, transform=True):
        if type(data_dict) is dict:
            self.data = data_dict
        self.keys = []
        for i in self.data.keys():
            self.keys.append(i)
        if(transform is True):
            self.transform = True
        else: 
            self.transform = False
            
    def __len__(self):
        return np.size(self.keys)
    
    def __getitem__(self, index):
        key = self.keys[index]
        label = re.split('/',key)[3].split('.')[0]
        image = self.data.get(key)
        if (self.transform is True):
            image = generic_transforms(image,resize=int(img_size*1.125), randomcrop=img_size, toTensor=True)   
        return image, label

In [ ]:
class generic_dataset_test(torch.utils.data.Dataset):
    #The dataset is saved as dictionary files on disks
    #Merge the dictionary files depending upon the size of RAM before passing in
    #Dictionary key: path of the file; item: Image file loaded as uint8 bytes using PIL Image
    
    import numpy as np
    import re
    
    def __init__(self, data_dict, transform=True):
        if type(data_dict) is dict:
            self.data = data_dict
        self.keys = []
        for i in self.data.keys():
            self.keys.append(i)
        if(transform is True):
            self.transform = True
        else: 
            self.transform = False
            
    def __len__(self):
        return np.size(self.keys)
    
    def __getitem__(self, index):
        key = self.keys[index]
        label = re.split('/',key)[3].split('.')[0]
        image = self.data.get(key)
        if (self.transform is True):
            image = generic_transforms(image, toTensor=True)   
        return image, label

In [ ]:
class generic_paired_dataset(torch.utils.data.Dataset):
    #The dataset is saved as dictionary files on disks
    #Merge the dictionary files depending upon the size of RAM before passing in
    #Dictionary key: path of the file; item: Image file loaded as uint8 bytes using PIL Image
    
    import numpy as np
    import re
    
    def __init__(self, data_dict, transform=True):
        if type(data_dict) is dict:
            self.data = data_dict
        self.keys = []
        for i in self.data.keys():
            self.keys.append(i)
        if(transform is True):
            self.transform = True
        else: 
            self.transform = False
            
    def __len__(self):
        return np.size(self.keys)
    
    def __getitem__(self, index):
        key = self.keys[index]
        label = re.split('/',key)[3].split('.')[0]
        image = self.data.get(key)
        if (self.transform is True):
            # split AB image into A and B
            height, width = image.shape[0], image.shape[1]
            w2 = int(width / 2)
            image_A = image[:,:w2,:]
            image_B = image[:,w2:,:]
            image_A = generic_transforms(image_A,resize=int(img_size*1.125), randomcrop=img_size, toTensor=False)   
            image_B = generic_transforms(image_B,resize=int(img_size*1.125), randomcrop=img_size, toTensor=False)   
        return (image_A,image_B), label

In [ ]:
def generic_transforms(image, resize = None, randomcrop = None, means = None, stds = None, hflip = False, toTensor = True):
    import torchvision.transforms.functional as TF
    import random
    from PIL import Image
    
    image = Image.fromarray(image)
    
    if(resize is not None):
        image = TF.resize(image, resize)

    if(randomcrop is not None):
        T = torchvision.transforms.transforms.RandomCrop(size=randomcrop)
        image = T.__call__(img=image)

    if(hflip is True):
        if(random.random() > 0.5):
            image = TF.hflip(image)
            
    if(toTensor is True):
        image = TF.to_tensor(image)
    else:
        image = np.array(image)
        
    if(means is not None and stds is not None):
        image = TF.normalize(image,means,stds)

    return image

In [ ]:
class generic_paired_dataset(torch.utils.data.Dataset):
    #The dataset is saved as dictionary files on disks
    #Merge the dictionary files depending upon the size of RAM before passing in
    #Dictionary key: path of the file; item: Image file loaded as uint8 bytes using PIL Image
    
    import numpy as np
    import re
    
    def __init__(self, data_dict, transform=True):
        if type(data_dict) is dict:
            self.data = data_dict
        self.keys = []
        for i in self.data.keys():
            self.keys.append(i)
        if(transform is True):
            self.transform = True
        else: 
            self.transform = False
            
    def __len__(self):
        return np.size(self.keys)
    
    def __getitem__(self, index):
        key = self.keys[index]
        label = re.split('/',key)[3].split('.')[0]
        image = self.data.get(key)
        if (self.transform is True):
            # split AB image into A and B
            height, width = image.shape[0], image.shape[1]
            w2 = int(width / 2)
            image_A = image[:,:w2,:]
            image_B = image[:,w2:,:]
            image_A = generic_transforms(image_A,resize=int(img_size*1.125), randomcrop=img_size, toTensor=False)   
            image_B = generic_transforms(image_B,resize=int(img_size*1.125), randomcrop=img_size, toTensor=False)   
        return (image_A,image_B), label

In [ ]:
# transform to be applied if using custom image folder, when dataset is large enough to not fit in memory
transforms_ = transforms.Compose([
    transforms.Resize(int(img_size*1.125)),
    transforms.RandomCrop(img_size), 
    transforms.RandomHorizontalFlip(0.5),
    transforms.ToTensor(),
])

In [ ]:
class Imagefolder_(torchvision.datasets.ImageFolder):
    def __init__(self, root='./',transform= transforms_):
        super(Imagefolder_, self).__init__(root, transform = transforms_)
        return

In [ ]:
'''dataset_train_A = Imagefolder_('./datasets/catsvsdogs/dogs/')
dataset_train_B = Imagefolder_('./datasets/catsvsdogs/cats/')'''

In [ ]:
'''dataloader_train_A = torch.utils.data.DataLoader(
    dataset_train_A,
    batch_size=batchsize,
    num_workers=2,
    shuffle=True,
    pin_memory=False,
    drop_last=True
)

dataloader_train_B = torch.utils.data.DataLoader(
    dataset_train_B,
    batch_size= batchsize,
    num_workers=2,
    shuffle=True,
    pin_memory=False,
    drop_last=True
)'''

In [ ]:
# for dataset apples2oranges
if(mode=='train'):
    data_dict_train_A = torch.load('./apples2oranges/trainA.pt')
    data_dict_train_B = torch.load('./apples2oranges/trainB.pt')

In [ ]:
# for dataset horse2zebra
if(mode=='train'):
    data_dict_train_A = torch.load('./horse2zebra/trainA.pt')
    data_dict_train_B = torch.load('./horse2zebra/trainB.pt')

In [ ]:
'''if(mode=='train'):
    data_dict_train_A = torch.load('./datasets/Imagenet/Black_bear/train.pt')
    data_dict_train_B = torch.load('./datasets/Imagenet/Polar_bear/train.pt')'''

In [ ]:
'''if(mode=='train'):
    data_dict_train_A = torch.load('./cityscapes/trainA.pt')
    data_dict_train_B = torch.load('./cityscapes/trainB.pt')'''

In [ ]:
'''if(mode=='test'):
    data_dict_test_A = torch.load('./cityscapes/testA.pt')
    data_dict_test_B = torch.load('./cityscapes/testB.pt')'''

In [ ]:
# execute in the training mode
if(mode=='train'):
    dataset_train_A = generic_dataset(
        data_dict=data_dict_train_A,
        transform=True
    )

    dataset_train_B = generic_dataset(
        data_dict=data_dict_train_B,
        transform=True
    )

    dataloader_train_A = torch.utils.data.DataLoader(
        dataset_train_A,
        batch_size=batchsize,
        num_workers=2,
        shuffle=True,
        pin_memory=False,
        drop_last=True
    )

    dataloader_train_B = torch.utils.data.DataLoader(
        dataset_train_B,
        batch_size= batchsize,
        num_workers=2,
        shuffle=True,
        pin_memory=False,
        drop_last=True
    )

In [ ]:
print(dataset_train_A.__len__(),dataset_train_B.__len__())

In [ ]:
#execute in the testing mode
assert(mode == 'test')

# Use one of the datasets for testing. Assumes the dataset is available in the .pt format
if(mode=='test'):
    #data_dict_test_A = torch.load('./apples2oranges/testA.pt')
    #data_dict_test_B = torch.load('./apples2oranges/testB.pt')
    data_dict_test_A = torch.load('./horse2zebra/testA.pt')
    data_dict_test_B = torch.load('./horse2zebra/testB.pt')

if(mode=='test'):
    dataset_test_A = generic_dataset_test(
        data_dict=data_dict_test_A,
        transform=True
    )

    dataset_test_B = generic_dataset_test(
        data_dict=data_dict_test_B,
        transform=True
    )

    dataloader_test_A = torch.utils.data.DataLoader(
        dataset_test_A,
        batch_size=batchsize,
        num_workers=2,
        shuffle=False,
        pin_memory=False,
        drop_last=True
    )

    dataloader_test_B = torch.utils.data.DataLoader(
        dataset_test_B,
        batch_size= batchsize,
        num_workers=2,
        shuffle=False,
        pin_memory=False,
        drop_last=True
    )

In [ ]:
class UnetSkipConnectionBlock(nn.Module):
    """Defines the Unet submodule with skip connection.
        X -------------------identity----------------------
        |-- downsampling -- |submodule| -- upsampling --|
    """

    def __init__(self, outer_nc, inner_nc, input_nc=None,
                 submodule=None, outermost=False, innermost=False, norm_layer=nn.BatchNorm2d, use_dropout=False):
        """Construct a Unet submodule with skip connections.

        Parameters:
            outer_nc (int) -- the number of filters in the outer conv layer
            inner_nc (int) -- the number of filters in the inner conv layer
            input_nc (int) -- the number of channels in input images/features
            submodule (UnetSkipConnectionBlock) -- previously defined submodules
            outermost (bool)    -- if this module is the outermost module
            innermost (bool)    -- if this module is the innermost module
            norm_layer          -- normalization layer
            user_dropout (bool) -- if use dropout layers.
        """
        super(UnetSkipConnectionBlock, self).__init__()
        self.outermost = outermost
        if type(norm_layer) == functools.partial:
            use_bias = norm_layer.func == nn.InstanceNorm2d
        else:
            use_bias = norm_layer == nn.InstanceNorm2d
        if input_nc is None:
            input_nc = outer_nc
        downconv = nn.Conv2d(input_nc, inner_nc, kernel_size=4,
                             stride=2, padding=1, bias=use_bias)
        downrelu = nn.LeakyReLU(0.2, True)
        downnorm = norm_layer(inner_nc)
        uprelu = nn.ReLU(True)
        upnorm = norm_layer(outer_nc)

        if outermost:
            upconv = nn.ConvTranspose2d(inner_nc * 2, outer_nc,
                                        kernel_size=4, stride=2,
                                        padding=1)
            down = [downconv]
            up = [uprelu, upconv, nn.Tanh()]
            model = down + [submodule] + up
        elif innermost:
            upconv = nn.ConvTranspose2d(inner_nc, outer_nc,
                                        kernel_size=4, stride=2,
                                        padding=1, bias=use_bias)
            down = [downrelu, downconv]
            up = [uprelu, upconv, upnorm]
            model = down + up
        else:
            upconv = nn.ConvTranspose2d(inner_nc * 2, outer_nc,
                                        kernel_size=4, stride=2,
                                        padding=1, bias=use_bias)
            down = [downrelu, downconv, downnorm]
            up = [uprelu, upconv, upnorm]

            if use_dropout:
                model = down + [submodule] + up + [nn.Dropout(0.5)]
            else:
                model = down + [submodule] + up

        self.model = nn.Sequential(*model)

    def forward(self, x):
        if self.outermost:
            return self.model(x)
        else:   # add skip connections
            return torch.cat([x, self.model(x)], 1)

In [ ]:
class UnetGenerator(nn.Module):
    """Create a Unet-based generator"""

    def __init__(self, input_nc, output_nc, num_downs, ngf=64, norm_layer=nn.BatchNorm2d, use_dropout=False):
        """Construct a Unet generator
        Parameters:
            input_nc (int)  -- the number of channels in input images
            output_nc (int) -- the number of channels in output images
            num_downs (int) -- the number of downsamplings in UNet. For example, # if |num_downs| == 7,
                                image of size 128x128 will become of size 1x1 # at the bottleneck
            ngf (int)       -- the number of filters in the last conv layer
            norm_layer      -- normalization layer

        We construct the U-Net from the innermost layer to the outermost layer.
        It is a recursive process.
        """
        super(UnetGenerator, self).__init__()
        # construct unet structure
        unet_block = UnetSkipConnectionBlock(ngf * 8, ngf * 8, input_nc=None, submodule=None, norm_layer=norm_layer, innermost=True)  # add the innermost layer
        for i in range(num_downs - 5):          # add intermediate layers with ngf * 8 filters
            unet_block = UnetSkipConnectionBlock(ngf * 8, ngf * 8, input_nc=None, submodule=unet_block, norm_layer=norm_layer, use_dropout=use_dropout)
        # gradually reduce the number of filters from ngf * 8 to ngf
        unet_block = UnetSkipConnectionBlock(ngf * 4, ngf * 8, input_nc=None, submodule=unet_block, norm_layer=norm_layer)
        unet_block = UnetSkipConnectionBlock(ngf * 2, ngf * 4, input_nc=None, submodule=unet_block, norm_layer=norm_layer)
        unet_block = UnetSkipConnectionBlock(ngf, ngf * 2, input_nc=None, submodule=unet_block, norm_layer=norm_layer)
        self.model = UnetSkipConnectionBlock(output_nc, ngf, input_nc=input_nc, submodule=unet_block, outermost=True, norm_layer=norm_layer)  # add the outermost layer

    def forward(self, input):
        """Standard forward"""
        return self.model(input)

In [ ]:
def get_norm_layer(norm_type='instance'):
    """Return a normalization layer
    Parameters:
        norm_type (str) -- the name of the normalization layer: batch | instance | none
    For BatchNorm, we use learnable affine parameters and track running statistics (mean/stddev).
    For InstanceNorm, we do not use learnable affine parameters. We do not track running statistics.
    """
    if norm_type == 'batch':
        norm_layer = functools.partial(nn.BatchNorm2d, affine=True, track_running_stats=True)
    elif norm_type == 'instance':
        norm_layer = functools.partial(nn.InstanceNorm2d, affine=False, track_running_stats=False)
    elif norm_type == 'none':
        norm_layer = lambda x: Identity()
    else:
        raise NotImplementedError('normalization layer [%s] is not found' % norm_type)
    return norm_layer

In [ ]:
def init_weights(net, init_type='normal', init_gain=0.02):
    """Initialize network weights.
    Parameters:
        net (network)   -- network to be initialized
        init_type (str) -- the name of an initialization method: normal | xavier | kaiming | orthogonal
        init_gain (float)    -- scaling factor for normal, xavier and orthogonal.

    We use 'normal' in the original pix2pix and CycleGAN paper. But xavier and kaiming might
    work better for some applications. Feel free to try yourself.
    """
    def init_func(m):  # define the initialization function
        classname = m.__class__.__name__
        if hasattr(m, 'weight') and (classname.find('Conv') != -1 or classname.find('Linear') != -1):
            if init_type == 'normal':
                init.normal_(m.weight.data, 0.0, init_gain)
            elif init_type == 'xavier':
                init.xavier_normal_(m.weight.data, gain=init_gain)
            elif init_type == 'kaiming':
                init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
            elif init_type == 'orthogonal':
                init.orthogonal_(m.weight.data, gain=init_gain)
            else:
                raise NotImplementedError('initialization method [%s] is not implemented' % init_type)
            if hasattr(m, 'bias') and m.bias is not None:
                init.constant_(m.bias.data, 0.0)
        elif classname.find('BatchNorm2d') != -1:  # BatchNorm Layer's weight is not a matrix; only normal distribution applies.
            init.normal_(m.weight.data, 1.0, init_gain)
            init.constant_(m.bias.data, 0.0)

    print('initialize network with %s' % init_type)
    net.apply(init_func)  # apply the initialization function <init_func>

In [ ]:
def init_net(net, init_type='normal', init_gain=0.02, gpu_ids=0):
    """Initialize a network: 1. register CPU/GPU device (with multi-GPU support); 2. initialize the network weights
    Parameters:
        net (network)      -- the network to be initialized
        init_type (str)    -- the name of an initialization method: normal | xavier | kaiming | orthogonal
        gain (float)       -- scaling factor for normal, xavier and orthogonal.
        gpu_ids (int list) -- which GPUs the network runs on: e.g., 0,1,2

    Return an initialized network.
    """
    device = torch.device("cuda:0")
    assert(torch.cuda.is_available())
    net.to(device)
    #net = torch.nn.DataParallel(net, gpu_ids)  # multi-GPUs
    init_weights(net, init_type, init_gain=init_gain)
    return net

In [ ]:
def define_G(netG, ngf=64, input_nc=3, output_nc=3, norm='batch', use_dropout=False, init_type='normal', init_gain=0.02, gpu_ids=[0]):
    """Create a generator

    Parameters:
        input_nc (int) -- the number of channels in input images
        output_nc (int) -- the number of channels in output images
        ngf (int) -- the number of filters in the last conv layer
        netG (str) -- the architecture's name: resnet_9blocks | resnet_6blocks | unet_256 | unet_128
        norm (str) -- the name of normalization layers used in the network: batch | instance | none
        use_dropout (bool) -- if use dropout layers.
        init_type (str)    -- the name of our initialization method.
        init_gain (float)  -- scaling factor for normal, xavier and orthogonal.
        gpu_ids (int list) -- which GPUs the network runs on: e.g., 0,1,2

    Returns a generator

    Our current implementation provides two types of generators:
        U-Net: [unet_128] (for 128x128 input images) and [unet_256] (for 256x256 input images)
        The original U-Net paper: https://arxiv.org/abs/1505.04597

        Resnet-based generator: [resnet_6blocks] (with 6 Resnet blocks) and [resnet_9blocks] (with 9 Resnet blocks)
        Resnet-based generator consists of several Resnet blocks between a few downsampling/upsampling operations.
        We adapt Torch code from Justin Johnson's neural style transfer project (https://github.com/jcjohnson/fast-neural-style).

    The generator has been initialized by <init_net>. It uses RELU for non-linearity.
    """
    net = None
    norm_layer = get_norm_layer(norm_type=norm)

    if netG == 'resnet_9blocks':
        net = ResnetGenerator(input_nc, output_nc, ngf, norm_layer=norm_layer, use_dropout=use_dropout, n_blocks=9)
    elif netG == 'resnet_6blocks':
        net = ResnetGenerator(input_nc, output_nc, ngf, norm_layer=norm_layer, use_dropout=use_dropout, n_blocks=6)
    elif netG == 'unet_128':
        net = UnetGenerator(input_nc, output_nc, 7, ngf, norm_layer=norm_layer, use_dropout=use_dropout)
    elif netG == 'unet_256':
        net = UnetGenerator(input_nc, output_nc, 8, ngf, norm_layer=norm_layer, use_dropout=use_dropout)
    else:
        raise NotImplementedError('Generator model name [%s] is not recognized' % netG)
    return init_net(net, init_type, init_gain, gpu_ids)

In [ ]:
class NLayerDiscriminator(nn.Module):
    """Defines a PatchGAN discriminator"""

    def __init__(self, input_nc=3, ndf=64, n_layers=3, norm_layer=nn.BatchNorm2d):
        """Construct a PatchGAN discriminator

        Parameters:
            input_nc (int)  -- the number of channels in input images
            ndf (int)       -- the number of filters in the last conv layer
            n_layers (int)  -- the number of conv layers in the discriminator
            norm_layer      -- normalization layer
        """
        super(NLayerDiscriminator, self).__init__()
        if type(norm_layer) == functools.partial:  # no need to use bias as BatchNorm2d has affine parameters
            use_bias = norm_layer.func == nn.InstanceNorm2d
        else:
            use_bias = norm_layer == nn.InstanceNorm2d

        kw = 4
        padw = 1
        sequence = [nn.Conv2d(input_nc, ndf, kernel_size=kw, stride=2, padding=padw), nn.LeakyReLU(0.2, True)]
        nf_mult = 1
        nf_mult_prev = 1
        for n in range(1, n_layers):  # gradually increase the number of filters
            nf_mult_prev = nf_mult
            nf_mult = min(2 ** n, 8)
            sequence += [
                nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=2, padding=padw, bias=use_bias),
                norm_layer(ndf * nf_mult),
                nn.LeakyReLU(0.2, True)
            ]

        nf_mult_prev = nf_mult
        nf_mult = min(2 ** n_layers, 8)
        sequence += [
            nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=1, padding=padw, bias=use_bias),
            norm_layer(ndf * nf_mult),
            nn.LeakyReLU(0.2, True)
        ]

        sequence += [nn.Conv2d(ndf * nf_mult, 1, kernel_size=kw, stride=1, padding=padw)]  # output 1 channel prediction map
        self.model = nn.Sequential(*sequence)

    def forward(self, input):
        """Standard forward."""
        return self.model(input)

In [ ]:
class PixelDiscriminator(nn.Module):
    """Defines a 1x1 PatchGAN discriminator (pixelGAN)"""

    def __init__(self, input_nc=3, ndf=64, norm_layer=nn.BatchNorm2d):
        """Construct a 1x1 PatchGAN discriminator

        Parameters:
            input_nc (int)  -- the number of channels in input images
            ndf (int)       -- the number of filters in the last conv layer
            norm_layer      -- normalization layer
        """
        super(PixelDiscriminator, self).__init__()
        if type(norm_layer) == functools.partial:  # no need to use bias as BatchNorm2d has affine parameters
            use_bias = norm_layer.func == nn.InstanceNorm2d
        else:
            use_bias = norm_layer == nn.InstanceNorm2d

        self.net = [
            nn.Conv2d(input_nc, ndf, kernel_size=1, stride=1, padding=0),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(ndf, ndf * 2, kernel_size=1, stride=1, padding=0, bias=use_bias),
            norm_layer(ndf * 2),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(ndf * 2, 1, kernel_size=1, stride=1, padding=0, bias=use_bias)]

        self.net = nn.Sequential(*self.net)

    def forward(self, input):
        """Standard forward."""
        return self.net(input)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

In [ ]:
class ImagePool():
    """This class implements an image buffer that stores previously generated images.

    This buffer enables us to update discriminators using a history of generated images
    rather than the ones produced by the latest generators.
    """

    def __init__(self, pool_size):
        """Initialize the ImagePool class

        Parameters:
            pool_size (int) -- the size of image buffer, if pool_size=0, no buffer will be created
        """
        self.pool_size = pool_size
        if self.pool_size > 0:  # create an empty pool
            self.num_imgs = 0
            self.images = []

    def query(self, images):
        """Return an image from the pool.

        Parameters:
            images: the latest generated images from the generator

        Returns images from the buffer.

        By 50/100, the buffer will return input images.
        By 50/100, the buffer will return images previously stored in the buffer,
        and insert the current images to the buffer.
        """
        if self.pool_size == 0:  # if the buffer size is 0, do nothing
            return images
        return_images = []
        
        for image in images:
            image = torch.unsqueeze(image.data, 0)
            if self.num_imgs < self.pool_size:   # if the buffer is not full; keep inserting current images to the buffer
                self.num_imgs = self.num_imgs + 1
                self.images.append(image)
                return_images.append(image)
            else:
                p = random.uniform(0, 1)
                if p > 0.5:  # by 50% chance, the buffer will return a previously stored image, and insert the current image into the buffer
                    random_id = random.randint(0, self.pool_size - 1)  # randint is inclusive
                    tmp = self.images[random_id].clone()
                    self.images[random_id] = image
                    return_images.append(tmp)
                else:       # by another 50% chance, the buffer will return the current image
                    return_images.append(image)
        return_images = torch.cat(return_images, 0)   # collect all the images and return
        return return_images

In [ ]:
def set_requires_grad(nets, requires_grad=False):
    """Set requies_grad=False for all the networks to avoid unnecessary computations
    Parameters:
        nets (network list)   -- a list of networks
        requires_grad (bool)  -- whether the networks require gradients or not
    """
    if not isinstance(nets, list):
        nets = [nets]
    for net in nets:
        if net is not None:
            for param in net.parameters():
                param.requires_grad = requires_grad

In [ ]:
class GANLoss(nn.Module):
    """Define different GAN objectives.
    The GANLoss class abstracts away the need to create the target label tensor
    that has the same size as the input.
    """

    def __init__(self, gan_mode, target_real_label=1.0, target_fake_label=0.0):
        """ Initialize the GANLoss class.

        Parameters:
            gan_mode (str) - - the type of GAN objective. It currently supports vanilla, lsgan, and wgangp.
            target_real_label (bool) - - label for a real image
            target_fake_label (bool) - - label of a fake image

        Note: Do not use sigmoid as the last layer of Discriminator.
        LSGAN needs no sigmoid. vanilla GANs will handle it with BCEWithLogitsLoss.
        """
        super(GANLoss, self).__init__()
        self.register_buffer('real_label', torch.tensor(target_real_label))
        self.register_buffer('fake_label', torch.tensor(target_fake_label))
        self.gan_mode = gan_mode
        if gan_mode == 'lsgan':
            self.loss = nn.MSELoss()
        elif gan_mode == 'vanilla':
            self.loss = nn.BCEWithLogitsLoss()
        elif gan_mode in ['wgangp']:
            self.loss = None
        else:
            raise NotImplementedError('gan mode %s not implemented' % gan_mode)

    def get_target_tensor(self, prediction, target_is_real):
        """Create label tensors with the same size as the input.

        Parameters:
            prediction (tensor) - - tpyically the prediction from a discriminator
            target_is_real (bool) - - if the ground truth label is for real images or fake images

        Returns:
            A label tensor filled with ground truth label, and with the size of the input
        """

        if target_is_real:
            target_tensor = self.real_label
        else:
            target_tensor = self.fake_label
        return target_tensor.expand_as(prediction)

    def __call__(self, prediction, target_is_real):
        """Calculate loss given Discriminator's output and grount truth labels.

        Parameters:
            prediction (tensor) - - tpyically the prediction output from a discriminator
            target_is_real (bool) - - if the ground truth label is for real images or fake images

        Returns:
            the calculated loss.
        """
        if self.gan_mode in ['lsgan', 'vanilla']:
            target_tensor = self.get_target_tensor(prediction, target_is_real)
            loss = self.loss(prediction, target_tensor)
        elif self.gan_mode == 'wgangp':
            if target_is_real:
                loss = -prediction.mean()
            else:
                loss = prediction.mean()
        return loss

In [ ]:
print(img_size)

In [ ]:
if(mode == 'train'):
    if(img_size == 128):
        arch = 'unet_128'
    elif(img_size == 256):
        arch = 'unet_256'
    
    net_adversarial_1 = NLayerDiscriminator()
    net_GA = define_G(netG = arch, norm='batch')
    net_GB = define_G(netG = arch, norm='batch')

    if cuda:
        net_adversarial_1.cuda()
        net_GA.cuda()
        net_GB.cuda()
else:
    if(mode == 'test'):
        arch = 'unet_128'
        net_GA = define_G(netG = arch, norm='batch')
        net_GB = define_G(netG = arch, norm='batch')
        
        if cuda:
            net_GA.cuda()
            net_GB.cuda()

In [ ]:
if((img_size == 256) and (mode == 'test')):
    arch = 'unet_256'
    
    net_GA = define_G(netG = arch, norm='batch')
    net_GB = define_G(netG = arch, norm='batch')

    if cuda:
        net_GA.cuda()
        net_GB.cuda()

In [ ]:
optimizer_GA = torch.optim.Adam(net_GA.parameters(), lr = lr, betas=(b1, b2))
optimizer_GB = torch.optim.Adam(net_GB.parameters(), lr = lr, betas=(b1, b2))
optimizer_adversarial_1 = torch.optim.Adam(net_adversarial_1.parameters(), lr = lr, betas=(b1, b2))

In [ ]:
# alternate optimizer
'''lr = 0.001
optimizer_GA = torch.optim.RMSprop(net_GA.parameters(), lr = lr)
optimizer_GB = torch.optim.RMSprop(net_GB.parameters(), lr = lr)
optimizer_adversarial_1 = torch.optim.RMSprop(net_adversarial_1.parameters(), lr = lr)'''

In [ ]:
# enter the date_time of the experiment to load the checkpoints
date_time = 'Tue Sep  8 13:13:39 2020' 

In [ ]:
# Load saved checkpoints
model_saved = torch.load('./checkpoints/'+exp_name+str(date_time)+'/net_GA_500.pth')
net_GA.load_state_dict(model_saved)
model_saved = torch.load('./checkpoints/'+exp_name+str(date_time)+'/net_GB_500.pth')
net_GB.load_state_dict(model_saved)
model_saved = torch.load('./checkpoints/'+exp_name+str(date_time)+'/net_adversarial_1_500.pth')
net_adversarial_1.load_state_dict(model_saved)

In [ ]:
# Load saved optimizers
optimizer_saved = torch.load('./checkpoints/'+exp_name+str(date_time)+'/optimizer_GA_500.pth')
optimizer_GA.load_state_dict(optimizer_saved)
optimizer_saved = torch.load('./checkpoints/'+exp_name+str(date_time)+'/optimizer_GB_500.pth')
optimizer_GB.load_state_dict(optimizer_saved)
optimizer_saved = torch.load('./checkpoints/'+exp_name+str(date_time)+'/optimizer_adversarial_1_500.pth')
optimizer_adversarial_1.load_state_dict(optimizer_saved)

In [ ]:
#gan_mode= 'wgangp'
gan_mode= 'vanilla'
criterionGAN = GANLoss(gan_mode).cuda()
criterionL1 = torch.nn.L1Loss().cuda()
criterionL2 = torch.nn.MSELoss().cuda()

In [ ]:
def display_all(imgs, save = False):
    images = imgs.detach().cpu().numpy()
    images = np.transpose(images, axes=(0,2,3,1))
    if(save):
        try:
            os.mkdir('Imgs')
        except:
            print('Folder already exists')
    for i in range(images.shape[0]):
        plt.imshow(images[i])
        if(save):
            plt.savefig('./Imgs/'+str(i)+'.png')
        else:
            plt.show()
    return

In [ ]:
def save_all(imgs, epoch, label, date, path= dir_checkpoint, experiment = exp_name):
    images = imgs.detach().cpu().numpy()
    images = np.transpose(images, axes=(0,2,3,1))
    plt.axis('off')
    try:
        os.mkdir(path+'/'+experiment+str(date)+'/'+str(epoch))
    except:
        pass
    for i in range(images.shape[0]):
        plt.imshow(images[i])
        plt.savefig(path+'/'+experiment+str(date)+'/'+str(epoch)+'/'+str(label)+'_'+ str(i)+'.png', bbox_inches='tight', pad_inches=0)
    plt.close('all')
    return

In [ ]:
print(net_GA)
print(net_GB)
print(net_adversarial_1)

In [ ]:
n_epochs = 750 # depends upon the dataset/task
resume = False
adversarial_stage = 50 #  hyperparameter decides when the stage-2 begins. This stage depends upon the dataset/task
save_epoch_freq = 10 # help='frequency of saving checkpoints'

In [ ]:
#Implementation using Unet architecture
#resume= True #h elp = set True to resume from the loaded checkpoints
cdate = str(time.ctime())
os.mkdir('./checkpoints/'+exp_name+cdate)
print('Training started: ',time.ctime())
start = time.time()

dataloader_len = dataset_train_A.__len__()
if (dataset_train_A.__len__() > dataset_train_B.__len__()):
    dataloader_len = dataset_train_B.__len__()
    
for epoch in range(n_epochs):
    
    start_epoch = time.time()

    batch_DA_loss = 0
    batch_DB_loss = 0
    batch_D_common_loss = 0    
    batch_GA_loss = 0
    batch_GB_loss = 0
    batch_D_common_GAN_1 = 0
    batch_D_common_GAN_2 = 0
    
    batch_GA_GAN_loss = 0
    batch_GB_GAN_loss = 0
    batch_GA_L1_real_loss = 0
    batch_GA_L1_fake_loss = 0
    
    batch_GB_L1_real_loss = 0
    batch_GB_L1_fake_loss = 0

    batch_GA_common_GAN = 0
    batch_GB_common_GAN = 0

    batch_loss_DA_fake_A1 = 0
    batch_loss_DA_real_A1 = 0
    batch_loss_DA_fake_B1 = 0
    batch_loss_DA_fake_B2 = 0

    batch_loss_DB_fake_A1 = 0
    batch_loss_DB_fake_A2 = 0
    batch_loss_DB_fake_B1 = 0
    batch_loss_DB_real_B1 = 0

    batch_GA_common_GAN_1 = 0
    batch_GA_common_GAN_2 = 0
    batch_GB_common_GAN_1 = 0
    batch_GB_common_GAN_2 = 0
    
    batch_loss_D_common_fake_A = 0
    batch_loss_D_common_real_A = 0
    batch_loss_D_common_fake_B = 0
    batch_loss_D_common_real_B = 0
    
    print('Starting epoch {}/{}.'.format(epoch + 1, n_epochs))

    
    real_images_A = torch.Tensor(batchsize, channels, img_size, img_size)
    fake_images_A = torch.Tensor(batchsize, channels, img_size, img_size)
    real_images_B = torch.Tensor(batchsize, channels, img_size, img_size)
    fake_images_B = torch.Tensor(batchsize, channels, img_size, img_size)
        
    ip_images_A = torch.Tensor(batchsize, channels, img_size, img_size)
    ip_images_B = torch.Tensor(batchsize, channels, img_size, img_size)
        
    dl_train_A = iter(dataloader_train_A)
    dl_train_B = iter(dataloader_train_B)
    
    for i in range(dataloader_len//batchsize):
        try:
            imgs_1, target_1 =  next(dl_train_A)
        except StopIteration:
            dl_train_A = iter(dataloader_train_A)
            imgs_1, target_1 =  next(dl_train_A)
            
        try:
            imgs_2, target_2 =  next(dl_train_B)
        except StopIteration:
            dl_train_B = iter(dataloader_train_B)
            imgs_2, target_2 =  next(dl_train_B)
                
        with torch.no_grad():
            
            for j in range(batchsize):
                # generate random numbers to select images of the other domain
                rand_num_1 = np.random.randint(batchsize)
                rand_num_2 = np.random.randint(batchsize)

                # prevent same images from appearing as real or fake
                while(rand_num_1==j):
                    rand_num_1 = np.random.randint(batchsize)
                while(rand_num_2==j):
                    rand_num_2 = np.random.randint(batchsize)
                
                real_images_A[j] = imgs_1[rand_num_1] # Domain A image. Real Image to Discriminator A e.g. Horse 
                fake_images_A[j] = imgs_2[rand_num_2] # Random Domain B image. Fake image to Discriminator A. e.g. Zebra
                
                real_images_B[j] = imgs_2[rand_num_1] # Domain B image, Real Image to Discriminator B e.g. Zebra 
                fake_images_B[j] = imgs_1[rand_num_2] # Random Domain B image. Fake image to Discriminator B. e.g. Horse

                ip_images_A[j] = imgs_2[j] # Input to the generator A is Domain B image. e.g. Zebra 
                ip_images_B[j] = imgs_1[j] # Input to the generator B is Domain A image. e.g. Horse
                                
            if(cuda):
                ip_images_A = ip_images_A.cuda()
                ip_images_B = ip_images_B.cuda()
                                
                real_images_A = real_images_A.cuda()
                fake_images_A = fake_images_A.cuda()
                                
                real_images_B = real_images_B.cuda() 
                fake_images_B = fake_images_B.cuda()

            # compute fake images:
            #ip_images_A = torch.cat((ip_images_A_1, ip_images_A_2), 2)
            #ip_images_B = torch.cat((ip_images_B_1, ip_images_B_2), 2)
    
        #--------------------------------------------------------------------
        #       Update Generator for Domain 1
        #--------------------------------------------------------------------
        
        optimizer_GB.zero_grad()        # set GB's gradients to zero
        optimizer_adversarial_1.zero_grad()        # set GA's gradients to zero
        
        # update GA
        set_requires_grad(net_adversarial_1, True)  # DB requires no gradients when optimizing GA
        set_requires_grad(net_GA, True)  # GA requires gradients when optimizing Generator for domain 1
        set_requires_grad(net_GB, False)  # GB requires no gradients when optimizing GA
        
        optimizer_GA.zero_grad()        # set GA's gradients to zero

        recon_images_A = net_GA(ip_images_A) # Construct the images from input images
        
        #Calculate GAN and L1 loss for the generator
        
        # Generator A should generate images that are not real for discriminator B 
        fake_A2 = torch.cat((ip_images_A, recon_images_A), 2)
        pred_fake_A2_DB = net_adversarial_1(fake_A2)
        loss_GA_GAN_DB = criterionGAN(pred_fake_A2_DB, True) * lambda_criterionGAN
                
        # Second, G(A) = B
        loss_GA_L1_real = criterionL1(recon_images_A, ip_images_A) * lambda_L1_real
                                
        # combine loss and calculate gradients
        loss_GA = (loss_GA_GAN_DB + loss_GA_L1_real) * 0.50
        loss_GA.backward()

        optimizer_GA.step()             # udpate G's weights
        
        batch_GA_GAN_loss += loss_GA_GAN_DB.item()
        
        batch_GA_L1_real_loss += loss_GA_L1_real.item()
        batch_GA_loss += loss_GA.item()
        
        
        #--------------------------------------------------------------------
        #       Update Generator for Domain 2
        #--------------------------------------------------------------------
        
        optimizer_GA.zero_grad()        # set GA's gradients to zero
        optimizer_adversarial_1.zero_grad()        # set GA's gradients to zero
        
        # update GB
        set_requires_grad(net_adversarial_1, True)  # DB requires no gradients when optimizing GB
        set_requires_grad(net_GA, False)  # GA requires no gradients when optimizing GB
        set_requires_grad(net_GB, True)  # GB requires gradients when optimizing GB
        
        optimizer_GB.zero_grad()        # set GB's gradients to zero

        recon_images_B = net_GB(ip_images_B)
        
        #Calculate GAN and L1 loss for the generator
                
        # First, G(A) should fake the discriminator
        fake_B2 = torch.cat((ip_images_B, recon_images_B), 2)
        pred_fake_B2_DB = net_adversarial_1(fake_B2)
        loss_GB_GAN_DB = criterionGAN(pred_fake_B2_DB, False) * lambda_criterionGAN
        
        # Second, G(A) = B
        loss_GB_L1_real = criterionL1(recon_images_B, ip_images_B) * lambda_L1_real
                                
        # combine loss and calculate gradients
        loss_GB = (loss_GB_GAN_DB + loss_GB_L1_real) * 0.50
        
        loss_GB.backward()

        optimizer_GB.step()             # udpate G's weights
        
        batch_GB_GAN_loss += loss_GB_GAN_DB.item()
        
        batch_GB_L1_real_loss += loss_GB_L1_real.item()
        
        batch_GB_loss += loss_GB.item()
        
        #--------------------------------------------------------------------
        #       Update Domain 2 Discriminator 
        #--------------------------------------------------------------------
        # realA real_images 
        # realB content images
        # fakeB recon_images
        
        optimizer_GA.zero_grad()        # set GA's gradients to zero
        optimizer_GB.zero_grad()        # set GA's gradients to zero
        
        # update DB
        set_requires_grad(net_adversarial_1, True)  # DB requires gradients when optimizing Discriminator for domain B
        set_requires_grad(net_GA, False)  # GA requires no gradients when optimizing DB
        set_requires_grad(net_GB, False)  # GB requires no gradients when optimizing DB
                
        # enable backprop for D
        optimizer_adversarial_1.zero_grad()     # set DB's gradients to zero

        #Calculate GAN loss for the discriminator

        # Domain B images are real for discriminator B
        real_A1 = torch.cat((ip_images_A, fake_images_A), 2)
        pred_real_A1_DB = net_adversarial_1(real_A1)
        loss_DB_real_A1 = criterionGAN(pred_real_A1_DB, False) * lambda_criterionGAN

        # Generator A images are fake for discriminator B        
        fake_A1 = torch.cat((ip_images_A, recon_images_A.detach()), 2)
        pred_fake_A1_DB = net_adversarial_1(fake_A1)
        loss_DB_fake_A1 = criterionGAN(pred_fake_A1_DB, False) * lambda_criterionGAN
        
        # Domain A images are fake for discriminator B
        fake_A2 = torch.cat((ip_images_A, real_images_A), 2)
        pred_fake_A2_DB = net_adversarial_1(fake_A2)
        loss_DB_fake_A2 = criterionGAN(pred_fake_A2_DB, True) * lambda_criterionGAN
        
        # Domain B images are real for discriminator B
        real_B1 = torch.cat((ip_images_B, real_images_B), 2)
        pred_real_B1_DB = net_adversarial_1(real_B1)
        loss_DB_real_B1 = criterionGAN(pred_real_B1_DB, False) * lambda_criterionGAN

        # Generator B images are fake for discriminator B
        fake_B1 = torch.cat((ip_images_B, recon_images_B.detach()), 2)
        pred_fake_B1_DB = net_adversarial_1(fake_B1)
        loss_DB_fake_B1 = criterionGAN(pred_fake_B1_DB, True) * lambda_criterionGAN

        # Domain A images are fake for discriminator B
        fake_B2 = torch.cat((ip_images_B, fake_images_B), 2)
        pred_fake_B2_DB = net_adversarial_1(fake_B2)
        loss_DB_fake_B2 = criterionGAN(pred_fake_B2_DB, True) * lambda_criterionGAN

        # combine loss and calculate gradients
        loss_DB = (loss_DB_real_A1 + loss_DB_real_B1 + loss_DB_fake_A1 + loss_DB_fake_A2 + loss_DB_fake_B1 + loss_DB_fake_B2) * (1.0/6.0)
        loss_DB.backward()
        optimizer_adversarial_1.step()          # update D's weights

        batch_loss_DB_fake_A1 += loss_DB_fake_A1.item()
        batch_loss_DB_fake_A2 += loss_DB_fake_A2.item()
        batch_loss_DB_fake_B1 += loss_DB_fake_B1.item()
        batch_loss_DB_real_B1 += loss_DB_fake_B1.item()
        
        batch_DB_loss += loss_DB.item()
        
        #--------------------------------------------------------------------
      
        
        if(i%50==0):
            recon_imgs = torch.cat((recon_images_A, recon_images_B),2)
            ip_imgs = torch.cat((ip_images_A, ip_images_B,),2)
            imgs_display = torch.cat((ip_imgs, recon_imgs),3)
            save_all(imgs_display,epoch,'real_recon_'+str(i),cdate)
        
    end_epoch = time.time()
    epoch_time_sec = (end_epoch - start_epoch)
    epoch_time_min = epoch_time_sec/60
    print('Epoch',epoch,'time: ',int(epoch_time_min),'min',int(epoch_time_sec)%60,'s')
    #print('D-loss:',batch_D_loss,'G_GAN-Loss:',batch_G_GAN_loss, 'G_L1-Loss:',batch_G_L1_loss, 'G_L2-Loss:',batch_G_L2_loss)
    #print('D-loss:',batch_D_loss,'G_GAN-Loss:',batch_G_GAN_loss, 'G_L1_1-Loss:',batch_G_L1_1_loss, 'G_L1_2-Loss:',batch_G_L1_2_loss)
    if(epoch % save_epoch_freq == 0):
        torch.save(net_GA.state_dict(), dir_checkpoint+'/'+exp_name+str(cdate)+'/net_GA_{}.pth'.format(epoch + 1))
        torch.save(net_GB.state_dict(), dir_checkpoint+'/'+exp_name+str(cdate)+'/net_GB_{}.pth'.format(epoch + 1))
        print('Checkpoint {} saved !'.format(epoch + 1))        
    if(epoch % save_epoch_freq*10 == 0):
        torch.save(net_adversarial_1.state_dict(), dir_checkpoint+'/'+exp_name+str(cdate)+'/net_adversarial_1_latest.pth')


In [ ]:
torch.save(net_adversarial_1.state_dict(), dir_checkpoint+'/'+exp_name+str(cdate)+'/net_adversarial_1_{}.pth'.format(epoch + 1))
torch.save(net_GA.state_dict(), dir_checkpoint+'/'+exp_name+str(cdate)+'/net_GA_{}.pth'.format(epoch + 1))
torch.save(net_GB.state_dict(), dir_checkpoint+'/'+exp_name+str(cdate)+'/net_GB_{}.pth'.format(epoch + 1))

In [ ]:
torch.save(optimizer_adversarial_1.state_dict(), dir_checkpoint+'/'+exp_name+str(cdate)+'/optimizer_adversarial_1_{}.pth'.format(epoch + 1))
torch.save(optimizer_GA.state_dict(), dir_checkpoint+'/'+exp_name+str(cdate)+'/optimizer_GA_{}.pth'.format(epoch + 1))
torch.save(optimizer_GB.state_dict(), dir_checkpoint+'/'+exp_name+str(cdate)+'/optimizer_GB_{}.pth'.format(epoch + 1))

In [ ]:
date_time

In [ ]:
#For testing
mode = 'test'
if(mode)=='test':
    n_epochs = 1
    save_ip_and_recon = False
    
cdate = str(time.ctime())
os.mkdir('./checkpoints/'+exp_name+cdate)
print('Training started: ',time.ctime())
start = time.time()

start_checkpoint = 501
end_checkpoint = 760
checkpoint_interval = 10

for checkpoint in range(start_checkpoint,end_checkpoint,checkpoint_interval):
    for i in range(2):
        test_recon_images = []
        if(i%2 == 0):
            model_saved = torch.load('./checkpoints/'+exp_name+str(date_time)+'/net_GA_'+str(checkpoint)+'.pth')
            net_GA.load_state_dict(model_saved)
            model = net_GA
            model.eval()
            dl_test = iter(dataloader_test_B)
            
        else:
            model_saved = torch.load('./checkpoints/'+exp_name+str(date_time)+'/net_GB_'+str(checkpoint)+'.pth')
            net_GB.load_state_dict(model_saved)
            model = net_GB
            model.eval()
            dl_test = iter(dataloader_test_A)
    
        start_epoch = time.time()
        
        for epoch in range(n_epochs):

            print('Starting Test')
            ip_images = torch.Tensor(batchsize, channels, img_size, img_size)

            iterate = True
            count = 0
            while(iterate):
                try:
                    imgs, target =  next(dl_test)
                    count += 1
                except StopIteration:
                    iterate = False

                with torch.no_grad():

                    for j in range(batchsize):
                        ip_images[j] = imgs[j] # Input to the generator A is Domain B image. e.g. Zebra 

                    if(cuda):
                        ip_images = ip_images.cuda()

                recon_images = model(ip_images)
                if(save_ip_and_recon):
                    imgs_display = torch.cat((ip_images, recon_images),2)
                else:
                    imgs_display = recon_images.detach()
                    test_recon_images.append(imgs_display)
                    
                save_all(imgs_display,str(checkpoint)+str(i),'real_recon_'+str(count),cdate)
            
            if(save_ip_and_recon):
                dataset = np.array(test_recon_images)
                dataset = dataset.reshape(-1,3,256,256)
                #dataset = dataset.transpose(0,3,1,2)
                np.save(str(checkpoint)+str(i),dataset)
            
            end_epoch = time.time()
            epoch_time_sec = (end_epoch - start_epoch)
            epoch_time_min = epoch_time_sec/60
            print('Epoch',checkpoint,'time: ',int(epoch_time_min),'min',int(epoch_time_sec)%60,'s')

            print('Complete !!')

In [ ]:
# load images from path
def pil_loader(path):
    return Image.open(path).convert('RGB')

In [ ]:
# convert dataset to dictionary files and save as pytorch files
def dataset_to_dict_AB(dataset):
    print('started',dataset)
    start_save = time.time()
    train_classes_A, train_images_A = dataset_memory_loader(dataset,ops=['trainA'])
    train_classes_B, train_images_B = dataset_memory_loader(dataset,ops=['trainB'])
    
    test_classes_A, test_images_A = dataset_memory_loader(dataset,ops=['testA'])
    test_classes_B, test_images_B = dataset_memory_loader(dataset,ops=['testB'])
        
    torch.save(dict(zip(train_classes_A,train_images_A)),str(dataset)+'_trainA.pt')
    torch.save(dict(zip(train_classes_B,train_images_B)),str(dataset)+'_trainB.pt')
    
    torch.save(dict(zip(test_classes_A,test_images_A)),str(dataset)+'_testA.pt')
    torch.save(dict(zip(test_classes_B,test_images_B)),str(dataset)+'_testB.pt')

    end_save = time.time()
    print(dataset,'completed in',int(end_save-start_save),'s')
    return

In [ ]:
# 
def dataset_memory_loader(task, ops, dir_data  = './'):
    for op in ops:
        images = []
        file_names = []
        path = dir_data+task+'/'+op
        print(path)
        count = 0
        for file in os.listdir(path):
            img_path = path+'/'+file
            file_names.append(img_path)
            img = pil_loader(img_path)
            img_int = np.array(img,dtype = 'uint8')
            images.append(img_int)
        count = count + 1
        return file_names, images

In [ ]:
dataset_to_dict_AB('horse2zebra')

In [ ]:
# diagnose network
def diagnose_network(net, name='network'):
    """Calculate and print the mean of average absolute(gradients)

    Parameters:
        net (torch network) -- Torch network
        name (str) -- the name of the network
    """
    mean = 0.0
    count = 0
    for param in net.parameters():
        if param.grad is not None:
            mean += torch.mean(torch.abs(param.grad.data))
            count += 1
    if count > 0:
        mean = mean / count
    print(name)
    print(mean)